In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import ipywidgets as widgets
import DLC_sqOF_v1 as DLCsq
import DLC_cyOF_v1 as DLCcy
import DLC_plot_v1 as DLCp
import ast
from pathlib import Path
from functools import reduce
from itertools import *
from scipy import stats

In [ ]:
datapath = '/Users/Documents/...'

In [ ]:
genotype_options = ['Choose', 'MitoPark', 'DAT', 'Anxa1']; genotype = 'None'
manipulation_options = ['Choose', '', 'Casp3', 'TeLC']; manipulation = 'None'
rig_options = ['Choose', 'SquareOF', 'Cylinder']; rig = 'None'
age_options = ['Choose', '', '8wks', '16wks', '24wks']; age = 'None'

drop_down1 = widgets.Dropdown(options = genotype_options, value='Choose', description='genotype:')
drop_down2 = widgets.Dropdown(options = manipulation_options, value='Choose', description='manipulation:')
drop_down3 = widgets.Dropdown(options = rig_options, value='Choose', description='rig:')
drop_down4 = widgets.Dropdown(options = age_options, value='Choose', description='age:')

def dropdown_handler_genotype(change):
    global genotype
    print(change.new); genotype = change.new 

def dropdown_handler_manipulation(change):
    global manipulation
    print(change.new); manipulation = change.new 

def dropdown_handler_rig(change):
    global rig
    print(change.new); rig = change.new 

def dropdown_handler_age(change):
    global age
    print(change.new); age = change.new 
    
drop_down1.observe(dropdown_handler_genotype, names='value')
drop_down2.observe(dropdown_handler_manipulation, names='value')
drop_down3.observe(dropdown_handler_rig, names='value')
drop_down4.observe(dropdown_handler_age, names='value')
display(drop_down1, drop_down2, drop_down3, drop_down4)

In [ ]:
midbrain = 'SNc'

if genotype == 'MitoPark':
    if age == '8wks':
        ID = ['22337','22339','21433','21434','22338','22341', 
              '19976','20631','21435','21436','22336','22340'] 
        Control_ID = '22337|22339|21433|21434|22338|22341' # Littermate controls
        Test_ID    = '19976|20631|21435|21436|22336|22340' # MitoPark mice        
    elif age == '16wks': 
        ID = ['18976','19974','20373','20758','24035','19975','20374','21425', 
              '18979','19973','20375','20376','20377','20759','21426','21635','24034','24036','24037','24038'] 
        Control_ID = '18976|19974|20373|20758|24035|19975|20374|21425' 
        Test_ID    = '18979|19973|20375|20376|20377|20759|21426|21635|24034|24036|24037|24038' 
    elif age == '24wks':
        ID = ['20767','21105','22076','22077','22842','23111','23245','18136','19717','21108','21896','22073','23112','23114','23244', 
              '18135','19716','20928','20930','21107','21893','22075','22841','23113']
        Control_ID = '20767|21105|22076|22077|22842|23111|23245|18136|19717|21108|21896|22073|23112|23114|23244' 
        Test_ID    = '18135|19716|20928|20930|21107|21893|22075|22841|23113' 
elif genotype == 'DAT' and manipulation == 'Casp3' and midbrain == 'SNc':
    ID = ['16400','16606','16762','16837','16841','17167','17260','17263','19268','19929']
    Control_ID = '16400|16606|16762|17167|19929' 
    Test_ID = '16837|16841|17260|17263|19268' 
elif genotype == 'Anxa1' and manipulation == 'TeLC':
    ID = ['16629','16643','17876','17877','20591','20594','29812','30465','30466','30470','30472','30473','30893','31129']
    Control_ID = '16629|16643|17876|17877|20591|20594' 
    Test_ID = '29812|30465|30466|30470|30472|30473|30893|31129' 
else:
    print('Choose genotype or rig above!')

In [ ]:
name1_af = '' # no name
DAY = 'D1DLC'

if genotype == 'MitoPark':
    experiment = rig + '_' + genotype + manipulation  
    name2_bf = rig + '_' + genotype + '/'

    if rig == 'Cylinder':
        analysis_results = 'Analysis_' + rig + '_' + genotype + '_' + age 
        name1_bf = DAY + '_resnet50_cylinder_modelJul23shuffle1_1030000_filtered' # rename the index name (shorter) at dataframe

        frame_start = 0;
        frame_end = 27000;
        pcutoff = 0.5;
        rearing_th = 350; 
        rearing_def_frame = 15;
        spatial_resolution = 4/246;

        animal = []
        for i in range(len(ID)):
            animal.append(genotype + '_' + ID[i] + '_' + DAY + '_resnet50_cylinder_modelJul23shuffle1_1030000_filtered') 
            DLCcy.DLC_cylinder_individual(datapath, experiment, animal[i], analysis_results, genotype, age, frame_start, frame_end, 
                                          pcutoff, rearing_th, rearing_def_frame, spatial_resolution)

        DLCcy.DLC_cylinder_summary(datapath, rig, analysis_results, DAY, genotype, manipulation, age, 
                                   frame_start, frame_end, pcutoff, rearing_th, rearing_def_frame, spatial_resolution, Control_ID, Test_ID, midbrain)       

    elif rig == 'SquareOF':
        analysis_results = 'Analysis_' + rig + '_' + genotype + '_' + age  # for only MitoPark
        name1_bf = DAY + '_resnet50_squareOF_modelApr11shuffle1_1030000_filtered'
        frame_start = 9000;
        frame_end   = 36000;               
        stop_th = 0.5; 
        stop_frame = 15;
        move_th = 0.5; 
        move_frame = 15;
        pcutoff = 0.95;
        speed_outlier = 30;
        spatial_resolution = 4/75;

        animal = []
        for i in range(len(ID)):
            animal.append(genotype + '_' + ID[i] + '_' + DAY + '_resnet50_squareOF_modelApr11shuffle1_1030000_filtered') 
            DLCsq.DLC_square_individual(datapath, experiment, animal[i], analysis_results, genotype, age, frame_start, frame_end, 
                                        stop_th, stop_frame, move_th, move_frame, pcutoff, speed_outlier, spatial_resolution)  

        DLCsq.DLC_square_summary(datapath, rig, analysis_results, DAY, genotype, manipulation, age, frame_start, frame_end, 
                                 stop_th, stop_frame, move_th, move_frame, pcutoff, speed_outlier, spatial_resolution, Control_ID, Test_ID, midbrain)          
else:
    experiment = rig + '_' + genotype + '_' + manipulation 
    name2_bf = rig + '_' + genotype + '_' + manipulation 
    
    if rig == 'Cylinder':
        analysis_results = 'Analysis_' + rig + '_' + genotype + '_' + manipulation  
        name1_bf = DAY + '_resnet50_cylinder_modelJul23shuffle1_1030000_filtered'
        frame_start = 0;
        frame_end = 27000;
        pcutoff = 0.95;
        rearing_th = 250; # sensor
        rearing_def_frame = 15;
        spatial_resolution = 4/246;

        animal = []
        for i in range(len(ID)):
            animal.append(genotype + '_' + ID[i] + '_' + DAY + '_resnet50_cylinder_modelJul23shuffle1_1030000_filtered') 
            DLCcy.DLC_cylinder_individual(datapath, experiment, animal[i], analysis_results, genotype, age, frame_start, frame_end, 
                                          pcutoff, rearing_th, rearing_def_frame, spatial_resolution)

        DLCcy.DLC_cylinder_summary(datapath, rig, analysis_results, DAY, genotype, manipulation, age, 
                                   frame_start, frame_end, pcutoff, rearing_th, rearing_def_frame, spatial_resolution, Control_ID, Test_ID, midbrain)

    elif rig == 'SquareOF':
        analysis_results = 'Analysis_' + rig + '_' + genotype + '_' + manipulation  
        name1_bf = DAY + '_resnet50_squareOF_modelApr11shuffle1_1030000_filtered'
        frame_start = 9000;
        frame_end   = 36000;                  

        stop_th = 0.5; 
        stop_frame = 15;
        move_th = 0.5; 
        move_frame = 15;
        pcutoff = 0.95;
        speed_outlier = 30;
        spatial_resolution = 4/75;

        animal = []
        for i in range(len(ID)):
            animal.append(genotype + '_' + ID[i] + '_' + DAY + '_resnet50_squareOF_modelApr11shuffle1_1030000_filtered')  
            DLCsq.DLC_square_individual(datapath, experiment, animal[i], analysis_results, genotype, age, frame_start, frame_end, 
                                        stop_th, stop_frame, move_th, move_frame, pcutoff, speed_outlier, spatial_resolution)  

        DLCsq.DLC_square_summary(datapath, rig, analysis_results, DAY, genotype, manipulation, age, 
                                 frame_start, frame_end, stop_th, stop_frame, move_th, move_frame, pcutoff, speed_outlier, spatial_resolution, Control_ID, Test_ID, midbrain)